## simple Gen Ai App with Langchain

In [25]:
## simple Gen Ai App with Langchain
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
##Langchain Tracking
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"]= os.getenv("LANGCHAIN_PROJECT")


In [26]:
## Data Ingestion -- from the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader


In [27]:
loader = WebBaseLoader("https://docs.smith.langchain.com/administration/concepts")
loader

In [28]:
docs = loader.load()
docs

[Document(page_content="\n\n\n\n\nConcepts | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationConceptual GuideOn this pageConcepts\nThis conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.\nResource Hierarchyâ€‹\nOrganizationsâ€‹\nAn organization is a logical grouping of

In [29]:
### Load Data --> Docs --> Divide our data into chunks --> texts --> vectors --> Vector Embeddings --> Vector Store DB
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documnets = text_splitter.split_documents(docs)

In [30]:
documnets

[Document(page_content='Concepts | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', metadata={'source': 'https://docs.smith.langchain.com/administration/concepts', 'title': 'Concepts | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.', 'language': 'en'}),
 Document(page_content='Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset trans

In [31]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [32]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documnets,embeddings)

In [33]:
vectorstoredb

In [34]:
#query from vectorstoredb
query = "LangSmith now has two tiers of traces based on Data Retention with the following characteristics:"
result = vectorstoredb.similarity_search(query)
result[0].page_content

'How it worksâ€‹\nLangSmith now has two tiers of traces based on Data Retention with the following characteristics:\nBaseExtendedPrice$.50 / 1k traces$5 / 1k tracesRetention Period14 days400 days\nData deletion after retention ends\nAfter the specified retention period, traces are no longer accessible via the runs table or API. All user data associated\nwith the trace (e.g. inputs and outputs) is deleted from our internal systems within a day thereafter. Some metadata\nassociated with each trace may be retained indefinitely for analytics and billing purposes.\nData retention auto-upgrades\ncautionAuto upgrades can have an impact on your bill. Please read this section carefully to fully understand your\nestimated LangSmith tracing costs.\nWhen you use certain features with base tier traces, their data retention will be automatically upgraded to\nextended tier. This will increase both the retention period, and the cost of the trace.'

In [35]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [36]:
## Retrieval Chain,Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following questions from the provided context:
<context>
{context}
</context>
    """
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following questions from the provided context:\n<context>\n{context}\n</context>\n    '))])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000262650B2BF0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002626512D210>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [37]:
from langchain_core.documents import Document
document_chain.invoke({
    "input": "After the specified retention period, traces are.",
    "context":[Document(page_content="After the specified retention period, traces are no longer accessible via the runs table or API. All user data associated with the trace (e.g. inputs and outputs) is deleted from our internal systems within a day thereafter. Some metadata associated with each trace may be retained indefinitely for analytics and billing purposes.")]
})

"Certainly! Please provide the questions you have about the context, and I'll do my best to answer them."

### However,we want the document to first come from the retriver we just setup.That way, we can use the retriver to dynamically select the most relevant documents and pass those in for a given question.

In [38]:
### Input --> Retriver --> vectordb --> LLM --> Output

vectorstoredb

In [40]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)


In [41]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000262652CAA70>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following questions from the provided context:\n<context>\n{context}\n</context>\n    '))])
            | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000262650B2BF0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions obj

In [42]:
### Get the response from llm 
response = retrieval_chain.invoke({ "input": "An organization is a logical grouping of users within LangSmith with its own billing configuration." })
response["answer"]

'Sure, here are answers to questions based on the provided context:\n\n1. **What is an organization in LangSmith?**\n   - An organization is a logical grouping of users within LangSmith with its own billing configuration. Typically, it represents one company and can have multiple workspaces.\n\n2. **What is the difference between a personal and a shared organization in LangSmith?**\n   - Personal organizations have a maximum of one workspace and cannot invite users, with the billing restricted to the developer plan. Shared organizations can have multiple workspaces (depending on the plan), allow user invitations, and have access to all other billing plans.\n\n3. **How are users managed in LangSmith?**\n   - Users, who are people with access to LangSmith, can be members of one or more organizations and workspaces. Organization members are managed in organization settings, and workspace members are managed in workspace settings.\n\n4. **What billing metrics are mentioned for LangSmith?**

In [43]:
response

{'input': 'An organization is a logical grouping of users within LangSmith with its own billing configuration.',
 'context': [Document(page_content="This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.\nResource Hierarchyâ€‹\nOrganizationsâ€‹\nAn organization is a logical grouping of users within LangSmith with its own billing configuration. Typically, there is one organization per company. An organization can have multiple workspaces. For more details, see the setup guide.\nWhen you log in for the first time, a personal organization will be created for you automatically. If you'd like to collaborate with others, you can create a separate organization and invite your team members to join.\nThere are a few important differences between your personal organization and shared organizations:\nFeaturePersonalSharedMaximum workspaces1Variable, depending on plan (see pricing pageCollaborationCannot invite usersCan invite usersBilling: p

In [44]:
response["context"]

[Document(page_content="This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.\nResource Hierarchyâ€‹\nOrganizationsâ€‹\nAn organization is a logical grouping of users within LangSmith with its own billing configuration. Typically, there is one organization per company. An organization can have multiple workspaces. For more details, see the setup guide.\nWhen you log in for the first time, a personal organization will be created for you automatically. If you'd like to collaborate with others, you can create a separate organization and invite your team members to join.\nThere are a few important differences between your personal organization and shared organizations:\nFeaturePersonalSharedMaximum workspaces1Variable, depending on plan (see pricing pageCollaborationCannot invite usersCan invite usersBilling: paid plansDeveloper plan onlyAll other plans available\nWorkspacesâ€‹", metadata={'source': 'https://docs.smith.langchain.com/